In [2]:
if (!require(survRM2)) install.packages('survRM2')
library('survRM2') 
if (!require(mvtnorm)) install.packages('mvtnorm')
library('mvtnorm')
if (!require(survival)) install.packages('survival')
library('survival')
if (!require(nph)) install.packages('nph')
library('nph')
if (!require(foreach)) install.packages("foreach")
library('foreach') 
if (!require(doParallel)) install.packages("doParallel")
library('doParallel')
if (!require(ggplot2)) install.packages("ggplot2")
library('ggplot2')
if (!require(cowplot)) install.packages("cowplot")
library('cowplot')
library('IRdisplay')
if (!require(abind)) install.packages("abind")
library('abind')

n_cores <- detectCores()
cluster <- makeCluster(16) # How many cores we use
registerDoParallel(cluster)
# multi thread
invisible(clusterEvalQ(cluster, #import packages to parallel 
  {
  library('survRM2')
  library('mvtnorm')
  library('survival')
  library('nph')
  library("simtrial")
  library("foreach")
  }))

source('/home/r27user6/RMST_Code/Function.R')
clusterExport(cluster, "expo_gen_2stages")

# 3 Methods Comparison Under Proportional Hazard

### 2 stages Rejection region of overall:
### 1. Log rank test
#### $\textcolor{lightgreen}{\frac{W_1}{\sigma_1}>c_1\ \&\ \frac{W}{\sigma}>c}$
### 2. Simple RMST Difference
#### $ \textcolor{lightgreen}{E(\tau_1) - C(\tau_1) > m_1\ \&\  E(\tau_2)-C(\tau_2)>m_2 }$
### 3. Our RMST Rejection method
#### $ \textcolor{lightgreen}{E(\tau_1)-C(\tau_1)>m_1\ \&\  E(\tau_1)>t_1\ \&\  E(\tau_2)-C(\tau_2)>m_2\ \&\  E(\tau_2)>t_2}$
--------

#### Different from the setting in Jung's 2017 table 1:
#### We set the total accrual time = 2 years and follow up 1 year
#### Recruit rate is 60(30 per arm) per year calculate the test power of 3 method controling $\alpha = 0.05$
#### Interim period we set at $c(1,\ 1.2,\ 1.4,\ 1.6,\ 1.8)$ year


In [20]:
sim_size <- 5000
acc_time <- 2
cen_time <- 1
delta <- 1.8
lambda_H1 <- 0.9
lambda_H0 <- 0.9 * delta
r <- 60  # patient recruit rate
acc_time <- 2
interim_list <- c(1, 1.2, 1.4, 1.6, 1.8)
n <- acc_time * r / 2  # total sample size per arm
set.seed(2024)

data_C <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 0, interim = interim_list)    
data_E_H0 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list)                    
data_E_H1 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H1, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list)

test_data <- foreach(i = 1 : length(interim_list), .combine = 'cbind', .packages = 'foreach') %dopar% 
    {
        interim <- interim_list[i]
        rmst_h0_int <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
        rmst_h1_int <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
        rmst_h0_fin <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
        rmst_h1_fin <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
        rmst_data <- rbind(rmst_h0_int, rmst_h1_int, rmst_h0_fin, rmst_h1_fin)
        #Log rank
        z_stats_h0_int <- log_rank_sim(data_C = data_C[ , c(2,3,1), i], 
                                data_E = data_E_H0[ , c(2,3,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h1_int <- log_rank_sim(data_C = data_C[ , c(2,3,1), i], 
                                data_E = data_E_H1[ , c(2,3,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h0_fin <- log_rank_sim(data_C = data_C[ , c(4,5,1), i], 
                                data_E = data_E_H0[ , c(4,5,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        z_stats_h1_fin <- log_rank_sim(data_C = data_C[ , c(4,5,1), i], 
                                data_E = data_E_H1[ , c(4,5,1), i], sim_size =  sim_size,
                            n = n, alpha = 0.05, sided = 'greater')$z_stats
        logrank_data <- rbind(z_stats_h0_int, z_stats_h1_int, z_stats_h0_fin, z_stats_h1_fin)
        return(list(rmst_data = rmst_data, logrank_data = logrank_data))
    }


## <span style="color:yellow">Grid search remarks:
### The searching range should be consistant under a specific experiment setting
###  1. Searching range of m (RMST difference):
#### $\qquad$ Lower bound slightly smaller than $E_1 - C_1|H_0\ $; Upper bound larger than $E_2 - C_2|H_1$
#### $\qquad$ upper bound is controled by m_low + search_times * search_step
### 2. Searching range of t (RMST of experiment group):
#### $\qquad$ t_low slightly smaller than $E_1\ $; t_up slightly larger than $E_2$
#### $\qquad$ search_step of t = (upper - lower) / searching times
### 3. Searching range of c (critical value of Log rank):
#### $\qquad$ Lower bound slightly smaller than $\frac{W_1}{\sigma_1}|H_0\ $; Upper bound larger than $\frac{W}{\sigma}|H_1$
### · <font color=LightSeaGreen>Control PET0, PET1 and overall alpha, search for the highest power
### Once the search_times is set, the running time is fixed. 

In [21]:
# search for parameter under different interim period
best_result <- foreach(i = 1 : length(interim_list), .combine = 'cbind', .packages = 'foreach') %dopar% 
    {   rmst_data <- test_data[,i]$rmst_data
        logrank_data <- test_data[,i]$logrank_data
        search_times <- 500

        #RMST
        m_low <- quantile((rmst_data[2,] - rmst_data[1,]), 0.1 )
        # Smallest RMST difference: interim under H0
        m_up <- quantile((rmst_data[8,] - rmst_data[7,]), 0.9 )
        # Largest RMST difference: final under H1
        search_step_rmst <- (m_up - m_low) / search_times
        t_low <- quantile((rmst_data[2,]), 0.1)
        # Smallest experiment RMST: interim under H0
        t_up <- quantile((rmst_data[8,]), 0.9)

        #Log rank
        c_low <- min(logrank_data)
        c_up <- max(logrank_data)
        search_step_lr <- (c_up - c_low) / search_times

        # Control PET0 and PET1
        tar_a1 <- 0.8      #Upper bound of 1-PET0
        tar_pow1_low <- 0.1  # lower bound of 1-PET1
        tar_alpha <- 0.05

        best_our_rmst <- find_m_t_RMST(m_low = m_low, t_low = t_low, t_up = t_up, rmst_data = rmst_data, 
                            search_times = search_times, search_step = search_step_rmst, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)

        best_simple_rmst <- find_m_t_RMST(m_low = m_low, t_low = -Inf, t_up = t_up, rmst_data = rmst_data, 
                            search_times = search_times, search_step = search_step_rmst, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)

        best_log_rank <- find_m_logrank(m_low = c_low, logrank_data = logrank_data, 
                            search_times = search_times, search_step = search_step_lr, tar_a1 = tar_a1, 
                            tar_pow1_low = tar_pow1_low, tar_a2 = tar_alpha, sim_size = sim_size)
        # best result in dataframe
        return(list(best_our_rmst = best_our_rmst, 
                    best_simple_rmst = best_simple_rmst,
                    best_log_rank = best_log_rank))
    }

In [22]:
best_result

result.1                                                                                        
best_our_rmst    -0.11856032, 0.38922503, 0.20120000, 0.00840000, -0.09399512, 0.72921410, 0.04940000, 0.98480000
best_simple_rmst -0.07625359, -Inf, 0.20060000, 0.01480000, 0.17349256, -Inf, 0.04860000, 0.93640000             
best_log_rank    NULL                                                                                            
                 result.2                                                                                        
best_our_rmst    -0.12379270, 0.42083100, 0.20080000, 0.00440000, -0.08805762, 0.72786275, 0.04920000, 0.98840000
best_simple_rmst -0.08393435, -Inf, 0.20100000, 0.01000000, 0.17308334, -Inf, 0.04900000, 0.94060000             
best_log_rank    NULL                                                                                            
                 result.3                                                                                        
best_our_rmst    -0.12957824, 0.44787872, 0.20500000, 0.00200000, -0.04164365, 0.72730862, 0.04900000, 0.99020000
best_simple_rmst -0.08286299, -Inf, 0.20160000, 0.00660000, 0.17407088, -Inf, 0.04860000, 0.93920000             
best_log_rank    NULL                                                                                            
                 result.4                                                                                        
best_our_rmst    -0.12351818, 0.46359468, 0.20360000, 0.00120000, -0.04161879, 0.72503021, 0.04940000, 0.99020000
best_simple_rmst -0.08393348, -Inf, 0.20200000, 0.00340000, 0.17404960, -Inf, 0.04860000, 0.93860000             
best_log_rank    NULL                                                                                            
                 result.5                                                                                        
best_our_rmst    -0.12472958, 0.47924806, 0.20320000, 0.00080000, -0.03599863, 0.72153112, 0.04840000, 0.99040000
best_simple_rmst -0.08305292, -Inf, 0.20020000, 0.00240000, 0.16969585, -Inf, 0.04940000, 0.94120000             
best_log_rank    -0.8425079, 0.2030000, 0.0022000, 1.6783691, 0.0488000, 0.8972000